# Reading networks from netzschleuder

[Run notebook in Google Colab](https://colab.research.google.com/github/pathpy/pathpy/blob/master/doc/tutorial/netzschleuder.ipynb)  
[Download notebook](https://github.com/pathpy/pathpy/raw/master/doc/tutorial/netzschleuder.ipynb)

The [netzschleuder](https://networks.skewed.de) repository is an online repository of more thn 100,000 networks maintained by [Tiago Peixoto](https://skewed.de/tiago). With `pathpy` you can directly read any network from the netzschleuder repository to analyze and visualize it.

In [ ]:
pip install git+git://github.com/pathpy/pathpy.git

In [ ]:
import pathpy as pp

from pprint import pprint

We can use the `list_netzschleuder_records` function to retrieve a list of all network data sets in the `netzschleuder` repository:

In [ ]:
datasets = pp.io.graphtool.list_netzschleuder_records()
pprint(datasets[:20])

The use of additional query parameters (e.g. looking for data with specific tags or returning full records with all attributes) is supported by keyword arguments. For instance, to return all social networks in the `netzschleuder` repository, you can write:

In [ ]:
datasets = pp.io.graphtool.list_netzschleuder_records(tags='Social')
pprint(datasets[50:70])

To retrieve detailed metadata from a specific record, we can use the following function:

In [ ]:
datasets = pp.io.graphtool.read_netzschleuder_record('karate')
pprint(datasets)

As we can see above, each data set (e.g. `karate`) in the netzschleuder database can have multiple networks (e.g. multiple measurements like `77` and `78` above). To read a specific network into a `pathpy` object, we have to specify both the name of the data set as well as the name of the network as follows:

In [ ]:
n = pp.io.graphtool.read_netzschleuder_network('karate', '77')
print(n)

In [ ]:
pp.plot(n)

To retrieve a list of temporal networks in the netzschleuder database, we can write: 

In [ ]:
pp.io.graphtool.list_netzschleuder_records(tag='Temporal')

To retrieve the full information on a specific record, we can again write:

In [ ]:
pp.io.graphtool.read_netzschleuder_record('sp_hypertext')

If there is only a single network in the data set (which then has the same name as the data set name), we can omit the network name. If we read a network where the edge_properties include a `time` attribute, a temporal network will be returned automatically.

In [ ]:
tn = pp.io.graphtool.read_netzschleuder_network('sp_hypertext', 'contacts')
print(tn)

If we do not want to retrieve a temporal network, we can use the `ignore_temporal` argument:

In [ ]:
n = pp.io.graphtool.read_netzschleuder_network('sp_hypertext', 'contacts', ignore_temporal=True)
print(n)

We may instead want to keep all information on the edges, either by returning a multi-edge network in which multiple edges between the same nodes are allowed, or by projecting the multiple observations to a numerical `weight` attribute of edges, where an edge weigt of `n` indicates that this specific edge has been observed `n` times. We can control this behavior using the additional parameter `mutliedges`:

In [ ]:
n = pp.io.graphtool.read_netzschleuder_network('sp_hypertext', 'contacts', ignore_temporal=True, multiedges=True)
print(n)